# Sargon's Pen Pals
This notebook will pull all the names (senders and people mentioned) from letters to Sargon II, who reigned over Assyria 722-705 BCE. These letters were published in the series *State Archives of Assyria* (SAA) volumes 1, 5, and 15 (1987, 1990, and 2001). Electronic versions of these volumes are found in the [ORACC](http://oracc.org) project *State Archives of Assyria online* ([SAAo](http://oracc.org/saao).

In the letters to Sargon (and to other Assyrian kings) the addressee is hardly ever mentioned by name. Instead, the letter opens with "to the king my lord". Simo Parpola, the main editor of the SAA series, assigned the letters to kings, based on his vast knowledge of the corpus.

The current notebook will use a network approach to evaluate these assignments, by using the names of a few individuals that are known to have been contemporaries of Sargon. If these people are mentioned in a letter, it is likely that the letter is to Sargon. We may also look at second or third degree relationships, to estimate the plausibility that a letter was indeed sent to Sargon.

# 1. Import Libraries

In [1]:
import pandas as pd
import zipfile
import json
import os
import errno
import tqdm
import requests

# 2. Create Directories
Create directories called `jsonzip` and `output`. If the directories already exist, do nothing.

For the code, see [Stack Overflow](http://stackoverflow.com/questions/18973418/os-mkdirpath-returns-oserror-when-directory-does-not-exist).

In [2]:
directories = ["jsonzip", "output"]
for d in directories:
    try:
        os.mkdir(d)
    except OSError as exc:
        if exc.errno !=errno.EEXIST:
            raise
        pass

# 3. Download the JSON ZIP files
For the code see ####

In [3]:
volumes = ["saa01", "saa05", "saa15"]
CHUNK = 16 * 1024
for v in volumes:
    url = "http://build-oracc.museum.upenn.edu/json/saao-" + v + ".zip"
    file = "jsonzip/saao" + v + ".zip"
    with requests.get(url, stream=True) as r:
        if r.status_code == 200:
            print("Downloading " + url + " saving as " + file)
            with open(file, 'wb') as f:
                for c in tqdm.tqdm(r.iter_content(chunk_size=CHUNK)):
                    f.write(c)
        else:
            print(url + " does not exist.")

293it [00:02, 119.25it/s]


280it [00:02, 106.62it/s]


322it [00:03, 86.46it/s] 


# 4. Extract Proper Nouns
Each of the ZIP files contains a file `gloss-qpn.json` which contains the glossary of proper nouns in that volume. This file is extracted and loaded in `json`. The extracted data are put in a list, each element of the list represents the proper nouns of one SAA volume.

In [4]:
json_l = []
for v in volumes:
    file = "jsonzip/saao" + v + ".zip"
    z = zipfile.ZipFile(file)
    filename = "saao/" + v + "/gloss-qpn.json"
    qpn = z.read(filename).decode('utf-8')         #read and decode the qpn glossary json file
    data_json = json.loads(qpn)  
    json_l.append(data_json)

# 5. Entries and Instances
The function `parse()` builds two DataFrames from different elements in the `gloss-qpn.json`. The key `entries` holds all the headwords (lemmas) in the glossary, with information such as Part of Speech (`pos`), original spelling, number of attestations, etc. The elements to be extracted for the Proper Noun DataFrame (`df_pn`) are Citation Form (`cf`), Part of Speech (`pos`), Guide Word (`gw`) and `xis`. The field `xis` holds an ID for this entry. Note that the `xis` ID is unique *within a project* and may well be duplicated in another project. Moreover, the `xis` is *not persistent*. After a new build of the project, the `xis` IDs will be realigned.

The key `instances` in `gloss-qpn.json` provides all the instances (in list form) of each headword, using the same `xis` field to identify the headword. The instance is referred to in the format PROJECT:ID_TEXT.ID_LINE.ID_WORD, for instance: `saao/saa01:P243567.9.1`. We can iterate through the field `xis` in `df_pn` to select the headwords that we need. We build a second DataFrame (`dinst_df`) with two columns: the `xis` ID and the text ID. The text ID can be extracted from the reference by taking the part between the colon and the first dot.

The two DataFarmes share the field `xis`. In the second DataFrame (`inst_df`) the same `xis` ID may appear multiple times (because the same name may appear in multiple texts, or multiple times in the same text). We can merge the two DataFrames with the `pandas` function `merge()`, merging on `xis` and using the keys from `inst_df`. The DataFrame that is returned will now have a row for each name instance, associated with a text ID (a P, Q, or X number). Each volume of [SAAo](http://oracc.org/saao) will return a separate DataFrame. 

In [5]:
def parse(data_json):
    entries = data_json["entries"]
    df_pn = pd.DataFrame(entries)
    df_pn = df_pn[["cf", "gw", "pos", "xis"]]
    df_pn = df_pn.loc[df_pn["pos"].isin(["PN", "RN"])]
    instances = data_json["instances"]
    l = []
    for i in df_pn["xis"]:
        for k in instances[i]:
            QPN = k.split(":")[1]
            QPN = QPN.split(".")[0]
            d = [i, QPN]
            l.append(d)
    inst_df = pd.DataFrame(l)
    inst_df.columns = ["xis", "id_text"]
    df = inst_df.merge(df_pn, on='xis', how='left')
    return df

# 6. Create and Concatenate the Lists of Name Instances
For each of the projects with Sargon letters in [SAAo](http://oracc.org/saao) the `json` data from its `gloss-qpn.json` are sent to the `parse()` function. This function refturns a DataFrame with all name instances, associated with text IDs. The code below collects those DataFrames in the list `pn_l`, concatenates them and then drops the filed `xis`. Since `xis` is project-specific, it has become meaningless in this stage

In [6]:
pn_l = []
for j in json_l:
    pns = parse(j)
    pn_l.append(pns)
df = pd.concat(pn_l)
df = df.drop("xis", axis = 1)
df

,id_text,cf,gw,pos
0,P313425,Abattu,1,PN
1,X900008,Abu-lešir,1,PN
2,X900008,Abu-lešir,1,PN
3,P334830,Adad-abuʾa,1,PN
4,P334209,Adad-ibni,1,PN
5,P334210,Adad-ibni,1,PN
6,P334568,Adad-ibni,1,PN
7,P313505,Adad-iriba,1,PN
8,X900008,Adad-isseʾa,1,PN
9,P334127,Adad-nuri,1,PN


# 7. Node List
The Node List is simply the list of all unique headwords. Add to the nodes list whether a name is known as a (Sargon-period) eponym. Save the nodes list as `nodes.csv` in the `output` directory.

In [7]:
n_df = df[["cf", "gw"]].copy()
n_df = n_df.drop_duplicates().reset_index(drop=True)
n_df.columns = ["label", "namesake"]

In [8]:
with open("csv/sargoneponyms.csv", mode="r", encoding="utf-8") as f:
    eponyms = pd.read_csv(f)
eponyms["eponym"] = True

In [9]:
nodes_df = n_df.merge(eponyms, on = "label", how="outer")
nodes_df["eponym"] = nodes_df["eponym"].fillna(False)
nodes_df["Id"] = nodes_df["label"].copy()
nodes_df

,label,namesake,eponym,Id
0,Abattu,1,False,Abattu
1,Abu-lešir,1,False,Abu-lešir
2,Adad-abuʾa,1,False,Adad-abuʾa
3,Adad-ibni,1,False,Adad-ibni
4,Adad-iriba,1,False,Adad-iriba
5,Adad-isseʾa,1,False,Adad-isseʾa
6,Adad-nuri,1,False,Adad-nuri
7,Adad-remanni,1,False,Adad-remanni
8,Adda-hati,1,False,Adda-hati
9,Adda-šumki,1,False,Adda-šumki


In [10]:
with open("output/nodes.csv", mode="w", encoding="utf-8") as w:
    nodes_df.to_csv(w, index=False)

# 8 Edge List
Transform the Pandas Dataframe into a simple list of list. In order to produce the edge list we use a loop within a loop. The first loop goes through all the items in the list (all names). For each name, it goes through the entire list again, to find items that match the same text ID (P number). This way, the routine finds all pairs of names that appear in each letter.

The secondary loop begins at the location of the index of the primary loop. This way, the edge A == B is not duplicated by the edge B == A (since the edges are undirected).

If there is a text ID match in the secondary loop, make a list that contains `id_text`, `source`, and `target` - this list represents a single edge. Add this list to the list of lists `edges`.

In [11]:
data = df.values.tolist()
edges = []
for idx, item in enumerate(data):
    textid = item[0]
    source = item[1]
    for idx_2, item_2 in enumerate(data[idx:len(data)]):
        if item[0] == item_2[0]:
            if not item[1] == item_2[1]:   # no SELF == SELF edges
                target = item_2[1]
                edge = [textid, source, target]
                edges.append(edge)  

The object `edges` is a list of lists that can be transformed again into a Dataframe. If the same name is mentioned multiple times in the same letter, that will create duplicate edges. Drop the duplicates.

In [12]:
edges_df = pd.DataFrame(edges, columns= ["id_text", "source", "target"])
edges_df = edges_df.drop_duplicates().reset_index(drop=True)
edges_df

,id_text,source,target
0,P313425,Abattu,Amiru
1,P313425,Abattu,Ammi-leti
2,P313425,Abattu,Bel-liqbi
3,P313425,Abattu,Qanni
4,X900008,Abu-lešir,Adad-isseʾa
5,X900008,Abu-lešir,Bel-duri
6,X900008,Abu-lešir,Bel-lešir
7,X900008,Abu-lešir,Šamaš-ahu-iddina
8,P334830,Adad-abuʾa,Aššur-belu-taqqin
9,P334830,Adad-abuʾa,Duri-Adad


# 9. Save Edge List
Save the edge list as `edges.csv` in the `output` directory.

In [13]:
with open("output/edges.csv", mode="w", encoding="utf-8") as w:
    edges_df.to_csv(w, index=False)